In [75]:
import pandas as pd
import numpy as np

In [51]:
ift_data = pd.read_excel('data/ift_data.xlsx')

In [52]:
ift_data.head()

,Gas,Water_content,time_minutes,volume ratio,IFT
0,CH4,0.0,0.000050,1.000000,25.08
1,CH4,0.0,14.998333,1.002618,25.12
2,CH4,0.0,30.000000,1.005236,25.16
3,CH4,0.0,45.000000,1.006108,25.17
4,CH4,0.0,60.000000,1.007853,25.21


In [53]:
ift_data.info

<bound method DataFrame.info of      Gas  Water_content  time_minutes  volume ratio    IFT
0    CH4            0.0      0.000050      1.000000  25.08
1    CH4            0.0     14.998333      1.002618  25.12
2    CH4            0.0     30.000000      1.005236  25.16
3    CH4            0.0     45.000000      1.006108  25.17
4    CH4            0.0     60.000000      1.007853  25.21
..   ...            ...           ...           ...    ...
499  CO2            0.7    150.000000      0.978155  19.93
500  CO2            0.7    165.000000      0.979369  19.95
501  CO2            0.7    180.000000      0.974515  19.87
502  CO2            0.7    195.000000      0.976942  19.92
503  CO2            0.7    210.000000      0.973301  19.86

[504 rows x 5 columns]>

In [54]:
#data = 
data = ift_data.iloc[:,:3]
y = ift_data.iloc[:,4]

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=123, shuffle=True)

In [56]:
X_test.columns

Index(['Gas', 'Water_content', 'time_minutes'], dtype='object')

In [57]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline


In [58]:
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

In [60]:
numeric_features = ['Water_content', 'time_minutes']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Gas']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
reg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LinearRegression())])


reg.fit(X_train, y_train)
print("model score: %.3f" % reg.score(X_test, y_test))

model score: 0.924


In [69]:
model_intercept = reg.named_steps['classifier'].intercept_
model_intercept

21.731370086302867

In [72]:
coeff_parameter = pd.DataFrame(reg.named_steps['classifier'].coef_,columns=['Coefficient'])
coeff_parameter

,Coefficient
0,0.195740
1,-0.209414
2,2.549298
3,-2.549298


In [73]:
y_pred = reg.predict(X_test)

In [80]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.4651894344742688
Mean Squared Error: 0.37745742564979706
Root Mean Squared Error: 0.6143756388804793
